In [1]:
import pandas as pd
import workdays
import datetime
from datetime import datetime, timedelta, date
from xbbg import blp
import pdblp
# https://data.bloomberglp.com/labs/sites/2/2013/12/blpapi-developers-guide-1.38.pdf
# https://data.bloomberglp.com/professional/sites/10/2017/03/BLPAPI-Core-Developer-Guide.pdf

In [46]:
FX0 = ['USD', 'GBP', 'HKD', 'CAD', 'AUD', 'EUR']
FX = [x + 'JPY BGN Curncy' for x in FX0 ]
FX = blp.bdp(tickers=FX, flds='px_last')
FX.index = [x[:3] for x in FX.index.tolist()]
FX

,px_last
AUD,104.0650
CAD,114.8940
EUR,170.2300
GBP,199.8840
HKD,20.0957
USD,156.9900


In [213]:
def BDP(L,YK):
    T = [L[i] + 'A ' + YK[i] for i in range(len(L))]
    T1 = [L[i] + '1 ' + YK[i] for i in range(len(L))]

    BDP = blp.bdp(tickers=T, flds=['name','currency','country_iso','exch_code','undl_spot_ticker']).loc[T,:]
    tmp = BDP['undl_spot_ticker'] + ' Index'
    tmp = blp.bdp(tickers=tmp, flds='country_iso').loc[tmp,:].iloc[:,0]
    BDP['risk'] = list(tmp)

    BDP.insert(0,'ticker', L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=['contract_value','volume_avg_20d','volatility_20d']).loc[T1,:]
    BDP1.columns = ['value/unit','volume','vola']
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)
    BDP = BDP.rename(columns={'currency':'crncy','country_iso':'exch','exch_code':'exch_code','undl_spot_ticker':'undl','contract_value':'value/unit'})
    
    tmp = []
    for i in range(len(L)):
        tmp  += [(blp.bdh(T1[i], 'px_last', '19991201', '20220101', Per='M' ).reset_index().iloc[0,0]).strftime('%Y%m%d')]
    BDP['initinal_t/d'] = tmp
    BDP = BDP.replace('19991231', '-')

    BDP['fx'] = 1.0
    BDP.loc[BDP['crncy'] != 'JPY', 'fx'] = BDP['crncy'].map(FX['px_last'])
    BDP['value/unit_JPY'] = (BDP['value/unit'] * BDP['fx']).round(0)
    BDP = BDP.drop('fx', axis=1)

    tmp = BDP['risk']
    BDP = BDP.drop('risk', axis=1)
    BDP.insert(3, 'risk', tmp)

    BDP['value/unit'] = BDP.pop('value/unit')
    BDP['value/unit_JPY'] = BDP.pop('value/unit_JPY')
    BDP['initinal_t/d'] = BDP.pop('initinal_t/d')

    for j in ['value/unit','value/unit_JPY','volume']:
        BDP[j] = BDP[j].apply('{:,.0f}'.format)
        
    BDP['vola'] = BDP['vola'].round()
    
    return BDP
    
L= ['ES','NQ','RTY','VG','CF','GX','EO','IB','SW','Z ','PT','XP','TP','HI','XU','JGS','TWT']
# L= ['ES','NQ','PT','VG','GX','Z ','PT','XP','TP','HI','XU','JGS','TWT']
YK = ['Index'] * len(L)

BDP = BDP(L,YK)
print('Equity Futures')
print(BDP.shape)
BDP

Equity Futures
(17, 12)


,ticker,name,crncy,risk,exch,exch_code,undl,volume,vola,value/unit,value/unit_JPY,initinal_t/d
0,ES,S&P500 EMINI FUT Jun24,USD,US,US,CME,SPX,"1,283,036",11.0,"266,075","41,771,114",-
1,NQ,NASDAQ 100 E-MINI Jun24,USD,US,US,CME,NDX,"546,482",14.0,"377,520","59,266,865",-
2,RTY,E-Mini Russ 2000 Jun24,USD,US,US,CME,RTY,"157,760",17.0,"103,825","16,299,487",20170731
3,VG,EURO STOXX 50 Jun24,EUR,EU,DE,EUX,SX5E,"594,296",9.0,"50,420","8,582,997",-
4,CF,CAC40 10 EURO FUT Jun24,EUR,FR,FR,EOP,CAC,"51,040",9.0,"80,780","13,751,179",-
5,GX,DAX INDEX FUTURE Jun24,EUR,DE,DE,EUX,DAX,"36,395",10.0,"469,000","79,837,870",-
6,EO,AMSTERDAM IDX FUT Jun24,EUR,NL,NL,EOE,AEX,"10,642",7.0,"183,390","31,218,480",-
7,IB,IBEX 35 INDX FUTR Jun24,EUR,ES,ES,MFM,IBEX,"11,422",14.0,"112,655","19,177,261",-
8,SW,MINI FTSE/MIB FUT Jun24,EUR,IT,IT,MIL,FTSEMIB,"4,736",11.0,"34,597","5,889,447",20040331
9,Z,FTSE 100 IDX FUT Jun24,GBP,GB,GB,ICF,UKX,"80,118",7.0,"83,335","16,657,333",-


In [214]:
BDP1 = BDP[['name','crncy','risk','value/unit_JPY']].copy()
for i in [100,300,500]:
    BDP1[str(i) + ' mil'] = BDP1['value/unit_JPY'].str.replace(',', '').astype(float) / (i * 10**6)
    BDP1[str(i) + ' mil'] = BDP1[str(i) + ' mil'].apply('{:,.1%}'.format)
BDP1

,name,crncy,risk,value/unit_JPY,100 mil,300 mil,500 mil
0,S&P500 EMINI FUT Jun24,USD,US,"41,771,114",41.8%,13.9%,8.4%
1,NASDAQ 100 E-MINI Jun24,USD,US,"59,266,865",59.3%,19.8%,11.9%
2,E-Mini Russ 2000 Jun24,USD,US,"16,299,487",16.3%,5.4%,3.3%
3,EURO STOXX 50 Jun24,EUR,EU,"8,582,997",8.6%,2.9%,1.7%
4,CAC40 10 EURO FUT Jun24,EUR,FR,"13,751,179",13.8%,4.6%,2.8%
5,DAX INDEX FUTURE Jun24,EUR,DE,"79,837,870",79.8%,26.6%,16.0%
6,AMSTERDAM IDX FUT Jun24,EUR,NL,"31,218,480",31.2%,10.4%,6.2%
7,IBEX 35 INDX FUTR Jun24,EUR,ES,"19,177,261",19.2%,6.4%,3.8%
8,MINI FTSE/MIB FUT Jun24,EUR,IT,"5,889,447",5.9%,2.0%,1.2%
9,FTSE 100 IDX FUT Jun24,GBP,GB,"16,657,333",16.7%,5.6%,3.3%


In [3]:
def BDP(L,YK):
    T = [L[i] + "A " + YK[i] for i in range(len(L))]
    T1 = [L[i] + "1 " + YK[i] for i in range(len(L))]
    
    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","fut_ctd_isin"]).loc[T,:]
    isin = BDP.loc[T,["fut_ctd_isin"]].dropna(how='all')
    isin = [i for i in isin["fut_ctd_isin"]]

    tmp0 = ["year","dur","yld"]
    tmp1 = ["mty_years","dur_adj_bid","yld_ytm_bid"]


    BDP1 = blp.bdp(tickers = [i+" Corp" for i in isin], flds=tmp1)
    
    for i in isin:
        for j in range(len(tmp0)):
            BDP.loc[BDP["fut_ctd_isin"]==i,tmp0[j]] = BDP1.loc[i+" Corp",tmp1[j]]

    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)
    

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    for j in ["contract_value","volume"]:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["year","dur","yld","vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(blp.bdh(T1[i], "px_last", "19991201", "20210101", Per="M").reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["initinal_t/d"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L = ["TU","FV","TY","US","WN","DU","OE","RX","UB","OAT","IK","G ","CN","XM","JB"]
# L = ["TU","FV"]
YK = ["Comdty"] * len(L)

print("Bond Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Bond Futures
(15, 13)


,ticker,name,crncy,exch,exch_code,fut_ctd_isin,year,dur,yld,contract_value,volume,vola,initinal_t/d
0,TU,US 2YR NOTE (CBT) Dec23,USD,US,CBT,US91282CFK27,1.97,1.87,5.11,"202,602","689,037",1.51,-
1,FV,US 5YR NOTE (CBT) Dec23,USD,US,CBT,US91282CGP05,4.43,4.00,4.62,"105,445","1,294,550",3.47,-
2,TY,US 10YR NOTE (CBT)Dec23,USD,US,CBT,US91282CHW47,6.93,5.94,4.52,"108,688","1,592,339",5.21,-
3,US,US LONG BOND(CBT) Dec23,USD,US,CBT,US912810QC53,15.89,11.19,4.57,"116,906","373,837",10.91,-
4,WN,US ULTRA BOND CBT Dec23,USD,US,CBT,US912810SF66,25.39,16.18,4.68,"122,594","261,544",14.46,20100129
5,DU,EURO-SCHATZ FUT Dec23,EUR,DE,EUX,DE000BU22023,1.98,1.88,3.26,"104,910","632,391",1.15,-
6,OE,EURO-BOBL FUTURE Dec23,EUR,DE,EUX,DE000BU25018,5.07,4.67,2.75,"115,870","649,094",3.47,-
7,RX,EURO-BUND FUTURE Dec23,EUR,DE,EUX,DE0001102606,8.89,8.06,2.69,"129,560","733,014",6.26,-
8,UB,EURO-BUXL 30Y BND Dec23,EUR,DE,EUX,DE0001102432,24.89,20.16,2.85,"125,640","90,972",13.84,-
9,OAT,Euro-OAT Future Dec23,EUR,DE,EUX,FR001400H7V7,9.66,8.16,3.29,"124,200","186,716",7.08,20120430


In [4]:
def BDP(L,YK):
    T = [L[i] + "A " + YK[i] for i in range(len(L))]
    T1 = [L[i] + "1 " + YK[i] for i in range(len(L))]

    BDP = blp.bdp(tickers=T, flds=["name","currency","country_iso","exch_code","undl_spot_ticker"]).loc[T,:]
    BDP.insert(0,"ticker", L)
    BDP = BDP.reset_index(drop=True)

    BDP1 = blp.bdp(tickers=T1, flds=["contract_value","volume_avg_20d","volatility_20d"]).loc[T1,:]
    BDP1.columns = ["contract_value","volume","vola"]
    BDP = pd.concat([BDP, BDP1.reset_index(drop=True)], axis=1)

    BDP = BDP.rename(columns={"currency":"crncy","country_iso":"exch","exch_code":"exch_code","undl_spot_ticker":"undl"})
    
    tmp = ["contract_value","volume"]
    for j in tmp:
        BDP[j] = BDP[j].apply("{:,.0f}".format)
        
    for j in ["vola"]:
        BDP[j] = round(BDP[j],2).values
        
    tmp = []
    for i in range(len(L)):
        tmp  += [(blp.bdh(T1[i], "px_last", "19991201", "20220101", Per="M" ).reset_index().iloc[0,0]).strftime("%Y%m%d")]
    BDP["initinal_t/d"] = tmp
    BDP = BDP.replace("19991231", "-")
    
    return BDP
    
L= ["CL","CO","GC"]
YK = ["Comdty"] * len(L)

print("Commodity Futures")
BDP = BDP(L,YK)
print(BDP.shape)
BDP

Commodity Futures
(3, 10)


,ticker,name,crncy,exch,exch_code,undl,contract_value,volume,vola,initinal_t/d
0,CL,WTI CRUDE FUTURE Nov23,USD,US,NYM,USCRWTIC,"90,030","297,850",14.00,-
1,CO,BRENT CRUDE FUTR Nov23,USD,GB,ICE,COY,"93,270","262,090",13.36,-
2,GC,GOLD 100 OZ FUTR Dec23,USD,US,CMX,NaN,"194,560","159,892",9.04,-


In [5]:
def BDP(L):
    T = ["CSBS" + i + " TMUQ Curncy" for i in L]  #Mitsubishi TTM
    tmp = ["px_last","volatility_60d","rsi_30d"]
    BDP = blp.bdp(tickers=T, flds=tmp).loc[T,:]
    BDP = BDP.loc[T,:]

    for i in tmp:
        BDP[i.lower()] = round(BDP[i.lower()].astype(float),2)
    BDP.insert(0, "currency", L )
    BDP.insert(1, "ticker", T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={"volatility_60d":"vola"})

    return BDP

L = ["USD","EUR","GBP","CAD","AUD","NZD","CHF","NOK","SEK","HKD","CNY"]

print("FX")
BDP = BDP(L)
print(BDP.shape)
BDP

FX
(11, 5)


,currency,ticker,px_last,vola,rsi_30d
0,USD,CSBSUSD TMUQ Curncy,147.67,8.82,59.96
1,EUR,CSBSEUR TMUQ Curncy,157.40,8.82,52.68
2,GBP,CSBSGBP TMUQ Curncy,181.41,8.69,48.73
3,CAD,CSBSCAD TMUQ Curncy,109.57,8.24,59.81
4,AUD,CSBSAUD TMUQ Curncy,94.63,8.92,52.32
5,NZD,CSBSNZD TMUQ Curncy,87.52,9.03,53.69
6,CHF,CSBSCHF TMUQ Curncy,163.19,8.41,51.47
7,NOK,CSBSNOK TMUQ Curncy,13.71,10.76,51.68
8,SEK,CSBSSEK TMUQ Curncy,13.22,10.20,48.24
9,HKD,CSBSHKD TMUQ Curncy,18.89,8.68,60.86


In [6]:
def BDP(L):
    T = [i + " Index" for i in L]   
    BDP = blp.bdp(tickers=T, flds=["Security_name","long_comp_name","Currency","volatility_260d"]).loc[T,:]
    BDP.insert(0, "ticker", T)
    BDP = BDP.reset_index(drop=True)
    BDP = BDP.rename(columns={"volatility_260d":"vola"})
    BDP["vola"] = round(BDP["vola"].astype(float),2)

    for i in range(len(BDP)):
        tmp = blp.bdh(BDP.loc[i,"ticker"], ["px_last","tot_return_index_gross_dvds"], "20000101", "20230301")
        
        BDP.loc[i,"PR"] = round(tmp.iloc[len(tmp)-1,0] / tmp.iloc[0,0] - 1,2) 
        BDP.loc[i,"TR"] = round(tmp.iloc[len(tmp)-1,1] / tmp.iloc[0,1] - 1,2) 

        if tmp.iloc[len(tmp)-1,1] - tmp.iloc[len(tmp)-1,0] == 0:
            BDP.loc[i,"PR=TR"] = "1"
        else:
            BDP.loc[i,"PR=TR"] = ""

        BDP.loc[i,"Initial_TD"] = tmp.index[0].strftime("%Y%m%d")
    BDP = BDP.replace("20000104", "-")
    return BDP
# https://www.jpmorgan.com/insights/research/index-research
# https://www.jpmorgan.com/content/dam/jpm/cib/complex/content/markets/index-research/Global-Index-Research-Product-Guide-2022.pdf#

In [7]:
print("Japan Stocks")
L = ["TPX","TPXDDVD"]
BDP(L)

Japan Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,TPX Index,Tokyo Stock Exchange Tokyo Sto,Tokyo Stock Exchange Tokyo Stock Price Index T...,JPY,14.14,0.16,0.75,,-
1,TPXDDVD Index,Topix Total Return Index JPY,Topix Total Return Index JPY,JPY,14.21,0.75,0.75,1,-


In [8]:
print("Developed Country Stocks")
L = ["MBKO","M0KO","MXKO","M2KO","MAKO","MSHJKOK","GJ124280","HJ124336"]
BDP(L)

Developed Country Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,MBKO Index,MSCI Kokusai JPY Price Index,MSCI Kokusai JPY Price Index,JPY,17.77,1.77,1.77,1,20000103
1,M0KO Index,MSCI Kokusai Gross Total Retur,MSCI Kokusai Gross Total Return JPY Index,JPY,17.77,3.77,3.77,1,20000103
2,MXKO Index,MSCI KOKUSAI Index,MSCI KOKUSAI Index,USD,15.67,1.06,2.63,,20000103
3,M2KO Index,MSCI Kokusai Gross Total Retur,MSCI Kokusai Gross Total Return USD Index,USD,15.68,2.59,2.59,1,20000103
4,MAKO Index,MSCI Kokusai JPY Net Index,MSCI Kokusai JPY Net Index,JPY,17.77,3.28,3.28,1,20000131
5,MSHJKOK Index,MSCI Hedged Indices In JPY Kok,MSCI Hedged Indices In JPY Kokusai,JPY,NaN,0.44,0.44,1,20000131
6,GJ124280 Index,MSCI Kokusai 100% Hedged to JP,MSCI Kokusai 100% Hedged to JPY Index,JPY,14.76,1.50,1.50,1,20000131
7,HJ124336 Index,MSCI Kokusai 100% Hedged to JP,MSCI Kokusai 100% Hedged to JPY,JPY,14.75,-0.00,-0.00,1,20210226


In [9]:
print("Emerging Markets Stocks")
L = ["GDUEEGF","NDUEEGF","HJ137561"]
BDP(L)

Emerging Markets Stocks


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,GDUEEGF Index,MSCI Daily TR Gross EM USD,MSCI Daily TR Gross EM USD,USD,15.60,2.68,2.68,1,20000103
1,NDUEEGF Index,MSCI Emerging Net Total Return,MSCI Emerging Net Total Return USD Index,USD,15.60,2.41,2.41,1,20000103
2,HJ137561 Index,MSCI EM (Emerging Markets) 100,MSCI EM (Emerging Markets) 100% Hedged to JPY ...,JPY,13.11,1.31,1.31,1,20000131


In [10]:
print("Developed Country REIT")
L = ["SREITWJJ","SREITJWJ"]
BDP(L)

Developed Country REIT


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,SREITWJJ Index,S&P Developed Ex Japan REIT JP,S&P Developed Ex Japan REIT JPY Index,JPY,20.92,2.24,2.24,1,20000103
1,SREITJWJ Index,S&P Developed Ex Japan REIT To,S&P Developed Ex Japan REIT Total Return JPY I...,JPY,20.94,9.12,9.12,1,20000103


In [11]:
print("Japan Bonds")
L = ["BPITTO01"]
BDP(L)

Japan Bonds


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,BPITTO01 Index,BPI Total Index,BPI Total Index,JPY,3.43,0.37,0.37,1,-


In [12]:
print("Developed Country Bond")
L = ["SBWGNJYC","SBWGNJYU"]
BDP(L)

Developed Country Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,SBWGNJYC Index,FTSE Non-JPY WGBI Curr-Hedged,FTSE Non-JPY WGBI Curr-Hedged JPY,JPY,6.54,0.46,0.46,1,20000103
1,SBWGNJYU Index,FTSE Non-JPY WGBI JPY,FTSE Non-JPY WGBI JPY,JPY,10.00,2.15,2.15,1,20000103


In [13]:
print("Emerging Market Bond / USD Bond")
L = ["JPGCUJCP","JPGCHJCP","JPEIDIVR","JPGCCOMP"]
# L = ["JPGCUJCP","JPGCHJCP","JPEIDIVR","JPEIPLUS","JPEMCOMP","GBIE1545"]
tmp = BDP(L)
tmp

Emerging Market Bond / USD Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD
0,JPGCUJCP Index,unhedged Return JPY,J.P. Morgan EMBI Global Diversified unhedged R...,JPY,11.42,1.10,1.10,1,20121001
1,JPGCHJCP Index,Hedged JPY,J.P. Morgan EMBI Global Diversified Hedged JPY,JPY,7.35,1.89,1.89,1,20000103
2,JPEIDIVR Index,JPMorgan Monthly EMBIs,J.P. Morgan EMBI Global Diversified Composite,USD,7.07,3.68,3.68,1,20000103
3,JPGCCOMP Index,Composite,J.P. Morgan EMBI Global Diversified Composite,USD,7.34,3.68,3.68,1,20000103


In [14]:
print("Emerging Market Bond / Local Currency Bond")
L = ["JGENDVUJ","JGENDVHJ","JGENGUJG","JGENGHJG","JGENVUJG","JGENVHJG"]
tmp = BDP(L)
tmp["Remarks"] = ["GBI EM Div"]*2+["GBI EM Global"]*2+["GBI EM Global Div"]*2
tmp

Emerging Market Bond / Local Currency Bond


,ticker,security_name,long_comp_name,currency,vola,PR,TR,PR=TR,Initial_TD,Remarks
0,JGENDVUJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Unhedged,JPY,10.95,1.73,1.73,1,20021231,GBI EM Div
1,JGENDVHJ Index,Divers,J.P. Morgan GBI-EM Diversified JPY Hedged,JPY,3.13,0.27,0.27,1,20030101,GBI EM Div
2,JGENGUJG Index,JPY,J.P. Morgan GBI-EM Global Composite Unhedged JPY,JPY,10.59,2.57,2.57,1,20011231,GBI EM Global
3,JGENGHJG Index,JPY,J.P. Morgan GBI-EM Global Composite Hedged JPY,JPY,1.33,0.53,0.53,1,20011231,GBI EM Global
4,JGENVUJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,10.35,2.04,2.04,1,20021231,GBI EM Global Div
5,JGENVHJG Index,JPY,J.P. Morgan GBI-EM Global Diversified Composit...,JPY,2.46,0.39,0.39,1,20021231,GBI EM Global Div
